In [3]:
import os
import cv2
import torch

from face_recognition.models.FaceNet import FaceNetResnet
from face_recognition.utils.data_augmentation import augment_and_normalize
from face_recognition.utils.constants import PRETRAINED_URL, PRETRAINED_MODEL_DIR, MODEL_DIR, TRAINED_WEIGHTS_DIR

In [4]:
model = FaceNetResnet()

In [6]:
torch.load(TRAINED_WEIGHTS_DIR, map_location=torch.device(device))['model_state_dict']

{'epoch': 19,
 'embedding_dimension': 256,
 'batch_size_training': 256,
 'model_state_dict': OrderedDict([('model.conv1.weight',
               tensor([[[[ 7.1340e-06,  5.4624e-06,  5.7424e-06,  ...,  5.4411e-06,
                           5.1229e-06,  5.1795e-06],
                         [ 6.1709e-06,  4.9949e-06,  4.8182e-06,  ...,  4.5402e-06,
                           4.2787e-06,  4.2450e-06],
                         [ 5.4931e-06,  4.4970e-06,  4.0636e-06,  ...,  4.1007e-06,
                           3.8376e-06,  3.9093e-06],
                         ...,
                         [ 4.9894e-06,  4.1910e-06,  4.0370e-06,  ...,  3.8493e-06,
                           3.6957e-06,  3.7850e-06],
                         [ 5.8532e-06,  4.9468e-06,  4.8620e-06,  ...,  4.5851e-06,
                           4.3467e-06,  4.4832e-06],
                         [ 8.2405e-06,  6.8014e-06,  6.7050e-06,  ...,  6.4528e-06,
                           5.8936e-06,  6.0354e-06]],
               
  

In [7]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model.load_state_dict(torch.load(TRAINED_WEIGHTS_DIR, map_location=torch.device(device))['model_state_dict'])

<All keys matched successfully>

In [4]:
tens = torch.from_numpy(img).unsqueeze(0).float()

In [5]:
augmented = augment_and_normalize(tens)

In [6]:
augmented[0].shape

torch.Size([1, 3, 224, 224])

In [ ]:
hparams = {'margin': 0.02, 'optimizer': 'adagrad', 'weight_decay': 1e-5, 'lr': 0.05}
backbone = FaceNetResnet(pretrained=True)
model = LightningFaceNet(hparams, backbone)

In [ ]:
model = LightningFaceNet.load_from_checkpoint(checkpoint, hparams=hparams, model=backbone)

In [ ]:
model

In [ ]:
trainer = pl.Trainer(
    gpus=1 if torch.cuda.is_available() else 0,
    max_epochs=3,
)

In [ ]:
from torch.utils.data import DataLoader

test_loader = DataLoader(lfw_set, batch_size=16, shuffle=True, num_workers=8)
trainer.test(model, test_loader)

In [ ]:
model_name = os.path.join(results_path, 'presentation_model.pth')

In [ ]:
torch.save(model.model.state_dict(), model_name)

In [ ]:
from face_recognition.models.FaceNet import load_weights

In [ ]:
model = load_weights(model_name)


In [ ]:
model

In [ ]:
enc = model(lfw_set[0][0].unsqueeze(0))

In [ ]:
enc.shape